In [103]:
def normalize_weights(weights, tolerance=1e-9):
    # Check if weights already sum to approximately 1
    total = np.sum(weights)
    if abs(total - 1.0) < tolerance:
        return weights
        
    # Normalize weights
    normalized_weights = weights / total
    
    # Ensure exact sum to 1 by adjusting the largest weight
    sum_diff = 1.0 - np.sum(normalized_weights)
    if abs(sum_diff) > 0:
        max_idx = np.argmax(normalized_weights)
        normalized_weights[max_idx] += sum_diff
        
    # Verify the sum is now exactly 1.0
    assert abs(np.sum(normalized_weights) - 1.0) < tolerance
    
    return normalized_weights

In [150]:
import gurobipy as GRB
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from gurobipy import GRB, quicksum

def CVaR_optimization(bond_data, scenario_returns, alpha=0.95):
    S, N = scenario_returns.shape
    print(S)
    # Create the model
    model = gp.Model("CVaR_Bond_Optimization")

    model.setParam('OutputFlag', 0)

    # Decision variables
    w = model.addVars(N, vtype=GRB.CONTINUOUS, lb=0, name="w")  # weights for bonds
    x = model.addVars(N, vtype=GRB.BINARY, name="x")  # binary selection of bonds
    VaR = model.addVar(vtype=GRB.CONTINUOUS, name="VaR")  # Value at Risk variable
    z = model.addVars(S, vtype=GRB.CONTINUOUS, lb=0, name="z")  # Auxiliary for CVaR
    
    # Objective function: Minimize CVaR risk (USING S)
    model.setObjective(
        VaR + (1 / (S * (1 - alpha))) * gp.quicksum(z[s] for s in range(S)),
        GRB.MINIMIZE
    )
    
    # Constrain the total weight
    model.addConstr(gp.quicksum(w[i] for i in range(N)) >= 0.99 , "SumToOneLower")
    model.addConstr(gp.quicksum(w[i] for i in range(N)) <= 1, "SumToOneUpper")

    # Extract expected returns from the DataFrame
    expected_returns = bond_data["ExpectedReturn"].values  # Assuming "ExpectedReturns" is a column in the DataFrame

# Add constraint in the model
    #for i in range(N):
        #if expected_returns[i] < 0:
            #model.addConstr(x[i] == 0, f"ExcludeNegativeReturn_{i}")

# Coupling weights and binary variables: w[i] = 0 if x[i] = 0
    for i in range(N):
        model.addConstr(w[i] <= x[i], f"WeightCoupling_{i}")
    
# Create array of equal weights first
    weights = np.array([1/75] * 75)  # Creates array of 75 elements each with 1/75

# Normalize to ensure they sum to exactly 1
    normalized_weights = normalize_weights(weights)

# Now use the normalized weight as benchmark
    BenchmarkWeight = normalized_weights[0]  # Since all weights are equal, can take any element

# Deviation from benchmark
    deviation_limit = 0.1
    epsilon = 1e-2  # small tolerance

    for i in range(N):
        model.addConstr(w[i] >= (BenchmarkWeight - deviation_limit*BenchmarkWeight) - epsilon, f"LowerBound_{i}")
        model.addConstr(w[i] <= (BenchmarkWeight + deviation_limit*BenchmarkWeight) + epsilon, f"UpperBound_{i}")

    #VaR constraint
    
    model.addConstr(VaR >= gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N) for s in range(S)) / S, "VaR_calculation")

    #CVaR constraint
    
    for s in range(S):  # Iterate through the scenarios
        model.addConstr(z[s] >= VaR - gp.quicksum(w[i] * scenario_returns[s, i] for i in range(N)), f"cvar_constraint_{s}")


    # Optimize the model
    model.optimize()

    print("It ran")
    
    if model.status == GRB.OPTIMAL:
        #print("Optimal solution found. List of all weights:")
        weights = [w[i].X for i in range(N)]  # Get the optimized weights for bonds
        return np.array(weights)

    if model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
        return None


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import gmean
import sympy as sp

In [151]:
pd.set_option('mode.chained_assignment', None)
import numpy as np
from scipy.optimize import linprog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gmean

data = pd.read_csv('bonds_w_exp_returns.csv')
# Check if model is valid and call the corresponding optimization function
data = data.sort_values(by='Date')
data['Date'] = pd.to_datetime(data['Date'])

model = 'cVaR'
constraints = []
# Initial budget to invest ($100,000)
initialVal = 100000  

# Length of investment period 
investPeriod = 1

# Identify the tickers and the dates
tickers = data['SecurityId'].unique()
dates = data['Date'].unique()

n = len(tickers)   # Number of bonds
NoPeriods = len(dates) // investPeriod  

# Preallocate space for portfolio weights (x0 will track turnover)
x = np.zeros([n, NoPeriods])
x0 = np.zeros([n, NoPeriods])

# Preallocate space for portfolio value, turnover, and transaction costs
currentVal = np.zeros([NoPeriods + 1, 1])
currentVal[0] = initialVal
portfolio_returns = np.zeros(NoPeriods)
turnover = np.zeros([NoPeriods, 1])
transaction_costs = np.zeros([NoPeriods, 1])

lookback_window = 126  # Past 126 days
start_period = lookback_window // investPeriod  # Determine the first valid period

for period in range(start_period, NoPeriods):  # Start at period 126
    print("Testing period:", period)
    current_date = dates[period]  # The current day
    # Get the last 126 unique trading days (from the 'Date' column)
    end_date = current_date  # Current date is the last day
    start_date = dates[period - lookback_window]  # Start date is 126 days earlier

    # Filter data to only include the unique trading dates
    trading_days_in_range = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]
    unique_trading_dates = trading_days_in_range['Date'].unique()

    if len(unique_trading_dates) < lookback_window:
        print(f"Skipping period {period}: not enough unique trading days.")
        continue
    
    # Now we use only the last 126 unique trading days
    # Sort the trading dates and slice the last 126 unique ones
    last_126_trading_days = unique_trading_dates[-lookback_window:]

    # Filter the original data to keep only rows from the last 126 unique trading days
    rolling_window_data = data[data['Date'].isin(last_126_trading_days)]

    # Pivot to create a matrix of daily returns (rows: days, columns: bonds)
    daily_returns_matrix = rolling_window_data.pivot(index='Date', columns='SecurityId', values='ExpectedReturn')

    # Drop any columns (bonds) with missing data
    daily_returns_matrix = daily_returns_matrix.dropna(axis=1)

    # Check if there are enough scenarios (days) and bonds
    if daily_returns_matrix.shape[0] < lookback_window or daily_returns_matrix.shape[1] < len(tickers):
        print(f"Skipping period {period}: insufficient historical data for bonds.")
        continue


    current_bonds = data[data['Date'] == current_date]
    # Use the daily returns matrix as `scenario_returns`
    scenario_returns = daily_returns_matrix.values  # Convert to NumPy array
    weights = CVaR_optimization(current_bonds, scenario_returns, alpha=0.95)
    # Store weights
    if weights is None:
        print("model returned nothing")
    
    x[:, period] = weights

    print(f"Weights: {weights}")
    
    # Portfolio calculations (as in your original code)
    portfolio_return = np.sum(weights * current_bonds['ExpectedReturn'])
    print("return: ", portfolio_return)
    currentVal[period + 1] = currentVal[period] + portfolio_return
    portfolio_returns[period] = portfolio_return

    # Turnover and transaction costs (as in your original code)
    turnover[period] = np.sum(np.abs(weights - x0[:, period])) / 2
    # Calculate transaction costs
    turnover_weights = np.abs(weights - x0[:, period])

    transaction_costs[period] = np.sum(turnover_weights * current_bonds['BidAskSpread'].values)
    currentVal[period + 1] -= transaction_costs[period]
    x0[:, period] = weights


excess_returns = portfolio_returns

# Calculate Sharpe ratio
SR = (gmean(excess_returns + 1) - 1) / excess_returns.std()

# Average turnover and cumulative transaction cost
avgTurnover = np.mean(turnover[1:])
total_transaction_cost = np.sum(transaction_costs)

print('Sharpe ratio: ', str(SR))
print('Avg. turnover: ', str(avgTurnover))
print('Total transaction costs: ', str(total_transaction_cost))

Testing period: 126
126
It ran
Weights: [0.02466667 0.02466667 0.002      0.02466667 0.002      0.02466667
 0.002      0.002      0.01468534 0.01742936 0.02466667 0.02466667
 0.02466667 0.02466667 0.02466667 0.02466667 0.02466667 0.02466667
 0.02466667 0.02466667 0.02466667 0.002      0.02466667 0.02466667
 0.02466667 0.002      0.02466667 0.02466667 0.002      0.002
 0.00401817 0.002      0.02466667 0.02466667 0.002      0.002
 0.002      0.002      0.002      0.02466667 0.002      0.002
 0.002      0.02466667 0.02466667 0.002      0.002      0.002
 0.002      0.002      0.002      0.02466667 0.002      0.002
 0.002      0.02466667 0.00423906 0.002      0.02466667 0.01629473
 0.002      0.002      0.002      0.02466667 0.02466667 0.002
 0.02466667 0.002      0.002      0.002      0.002      0.02466667
 0.02466667 0.02466667 0.02466667]
return:  -0.7842917665950841
Testing period: 127
126
It ran
Weights: [0.02466667 0.02466667 0.002      0.02466667 0.002      0.02466667
 0.002      0.0

/Users/alexlugard/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:197: RuntimeWarning: invalid value encountered in log
  log_a = np.log(a)
